In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:

train = pd.read_csv('C:\\Users\\user\\workspace\\movie\\movies_train.csv')
test = pd.read_csv('C:\\Users\\user\\workspace\\movie\movies_test.csv')
submission = pd.read_csv('C:\\Users\\user\\workspace\\movie\\submission.csv')

전처리

In [5]:
train['distributor'] = train.distributor.str.replace("(주)", '')
test['distributor'] = test.distributor.str.replace("(주)", '')

In [6]:
train['distributor'] = [re.sub(r'[^0-9a-zA-Z가-힣]', '', x) for x in train.distributor]
test['distributor'] = [re.sub(r'[^0-9a-zA-Z가-힣]', '', x) for x in test.distributor]

In [7]:
def get_dis(x) :
    if 'CJ' in x or 'CGV' in x :
        return 'CJ'
    elif '쇼박스' in x :
        return '쇼박스'
    elif 'SK' in x :
        return 'SK'
    elif '리틀빅픽' in x :
        return '리틀빅픽처스'
    elif '스폰지' in x :
        return '스폰지'
    elif '싸이더스' in x :
        return '싸이더스'
    elif '에이원' in x :
        return '에이원'
    elif '마인스' in x :
        return '마인스'
    elif '마운틴픽' in x :
        return '마운틴픽처스'
    elif '디씨드' in x :
        return '디씨드'
    elif '드림팩트' in x :
        return '드림팩트'
    elif '메가박스' in x :
        return '메가박스'
    elif '마운틴' in x :
        return '마운틴'
    else :
        return x

In [8]:
train['distributor'] = train.distributor.apply(get_dis)
test['distributor'] = test.distributor.apply(get_dis)

In [9]:
train.groupby('genre').box_off_num.mean().sort_values()

genre
뮤지컬       6.627000e+03
다큐멘터리     6.717226e+04
서스펜스      8.261100e+04
애니메이션     1.819267e+05
멜로/로맨스    4.259680e+05
미스터리      5.275482e+05
공포        5.908325e+05
드라마       6.256898e+05
코미디       1.193914e+06
SF        1.788346e+06
액션        2.203974e+06
느와르       2.263695e+06
Name: box_off_num, dtype: float64

In [10]:
train['genre_rank'] = train.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})
test['genre_rank'] = test.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})

In [11]:
tr_nm_rank = train.groupby('distributor').box_off_num.median().reset_index(name = 'num_rank').sort_values(by = 'num_rank')
tr_nm_rank

,distributor,num_rank
110,인피니티엔터테인먼트,2.0
15,고구마공작소,8.0
52,사람과사람들,42.0
97,위드시네마,46.0
19,나우콘텐츠,54.0
...,...,...
113,전망좋은영화사,1214237.0
105,이십세기폭스코리아,1422844.0
56,쇼박스,2138560.0
84,영구아트무비,2541603.0


In [12]:
tr_nm_rank['num_rank'] = [i + 1 for i in range(tr_nm_rank.shape[0])]

In [13]:
tr_nm_rank

,distributor,num_rank
110,인피니티엔터테인먼트,1
15,고구마공작소,2
52,사람과사람들,3
97,위드시네마,4
19,나우콘텐츠,5
...,...,...
113,전망좋은영화사,143
105,이십세기폭스코리아,144
56,쇼박스,145
84,영구아트무비,146


데이터 병합

In [14]:
train = pd.merge(train, tr_nm_rank, how = 'left')

In [15]:
test = pd.merge(test, tr_nm_rank, how = 'left')

In [16]:
test.fillna(0, inplace = True)

In [20]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from ngboost import NGBRegressor

모델링 데이터 구성

In [21]:
X = train[['num_rank', 'time', 'num_staff', 'num_actor', 'genre_rank', 'screening_rat']]
y = np.log1p(train.box_off_num)

In [22]:
X = pd.get_dummies(columns = ['screening_rat'], data = X)

In [23]:
X['num_actor'] = np.log1p(X['num_actor'])

In [24]:
target = test[['num_rank', 'time', 'num_staff', 'num_actor', 'genre_rank', 'screening_rat']]

In [25]:
target = pd.get_dummies(columns = ['screening_rat'], data = target)

In [26]:
target['num_actor'] = np.log1p(target['num_actor'])

In [27]:
kf = KFold(n_splits =  10, shuffle = True, random_state = 42)

모델

In [28]:
gbm = GradientBoostingRegressor(random_state = 42)

In [29]:
rmse_list = []
gb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    gbm.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in gbm.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in gbm.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    gb_pred += (sub_pred / 10)

In [30]:
np.mean(rmse_list)

1174378.9765191542

In [31]:
ngb = NGBRegressor(random_state = 518)

In [32]:
rmse_list = []
ngb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    ngb.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in ngb.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in ngb.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    ngb_pred += (sub_pred / 10)

[iter 0] loss=2.6200 val_loss=0.0000 scale=1.0000 norm=2.9523
[iter 100] loss=1.9566 val_loss=0.0000 scale=2.0000 norm=2.7106
[iter 200] loss=1.5812 val_loss=0.0000 scale=2.0000 norm=2.1869
[iter 300] loss=1.4423 val_loss=0.0000 scale=2.0000 norm=2.0889
[iter 400] loss=1.3728 val_loss=0.0000 scale=1.0000 norm=1.0177
[iter 0] loss=1.4112 val_loss=0.0000 scale=1.0000 norm=1.0787
[iter 100] loss=1.3275 val_loss=0.0000 scale=1.0000 norm=1.0096
[iter 200] loss=1.2866 val_loss=0.0000 scale=2.0000 norm=1.9741
[iter 300] loss=1.2496 val_loss=0.0000 scale=0.5000 norm=0.4837
[iter 400] loss=1.2162 val_loss=0.0000 scale=1.0000 norm=0.9517
[iter 0] loss=1.2152 val_loss=0.0000 scale=1.0000 norm=0.9572
[iter 100] loss=1.1614 val_loss=0.0000 scale=0.5000 norm=0.4594
[iter 200] loss=1.1496 val_loss=0.0000 scale=0.1250 norm=0.1138
[iter 300] loss=1.1434 val_loss=0.0000 scale=0.0625 norm=0.0566
[iter 400] loss=1.1411 val_loss=0.0000 scale=0.0078 norm=0.0071
[iter 0] loss=1.1749 val_loss=0.0000 scale=1.0

In [33]:
np.mean(rmse_list)

1450172.9838715226

In [34]:
lgbm = LGBMRegressor(random_state = 518)

In [35]:
rmse_list = []
lgb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in lgbm.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in lgbm.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    lgb_pred += (sub_pred / 10)

In [36]:
np.mean(rmse_list)

1213815.0440528719

In [37]:
xgb = XGBRegressor(random_state = 518)

In [38]:
rmse_list = []
xgb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    xgb.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in xgb.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in xgb.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    xgb_pred += (sub_pred / 10)

In [39]:
np.mean(rmse_list)

1437735.1546705803

In [40]:
cat = CatBoostRegressor(random_state = 518, silent = True)

In [41]:
rmse_list = []
cat_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    cat.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in cat.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in cat.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    cat_pred += (sub_pred / 10)

In [42]:
np.mean(rmse_list)

1075600.9209088846

In [43]:
rf = RandomForestRegressor(random_state = 518)

In [44]:
rmse_list = []
rf_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    rf.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in rf.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in rf.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    rf_pred += (sub_pred / 10)

In [45]:
np.mean(rmse_list)

872738.262092494

In [46]:
submission['box_off_num'] = (xgb_pred + cat_pred + lgb_pred + rf_pred + gb_pred + ngb_pred) / 6

In [47]:
submission.sort_values(by = 'box_off_num')

,title,box_off_num
130,댄서김의 은밀한 교수법,1.220413e+01
39,REC 알이씨,1.472818e+01
139,화려한 외출,1.980938e+01
173,옹녀뎐,2.054573e+01
65,엄마는 창녀다,2.678864e+01
...,...,...
135,용의자,4.289062e+06
229,베테랑,4.346994e+06
142,박수건달,5.461635e+06
179,군도: 민란의 시대,6.625242e+06


In [55]:
submission

,title,box_off_num
0,용서는 없다,1.214802e+06
1,아빠가 여자를 좋아해,1.862881e+06
2,하모니,8.344196e+05
3,의형제,2.098653e+06
4,평행 이론,2.175617e+06
...,...,...
238,해에게서 소년에게,9.316682e+03
239,울보 권투부,3.877889e+03
240,어떤살인,1.257419e+02
241,말하지 못한 비밀,1.540487e+03


In [56]:
submission.to_csv('suhykii_first_movies.csv',index = False)

In [57]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'suhykii_first_movies.csv', 
'f6ad2d1aa34753fbc5c9c5288933d8c895954de18de4b3bd09191c03d29e2c60', 
'235536', 
'egm', 
'RandomForestRegressor' )

{'isSubmitted': True, 'detail': 'Success'}
